In [1]:
import numpy as np
import pandas as pd
import sympy as sp
import unittest
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [2]:
class Laguerre:
    def __init__(self, T, N, beta, sigma, epsilon, t_values):
        self.T = T
        self.N = N
        self.beta = beta
        self.sigma = sigma
        self.epsilon = epsilon
        self.t_values = t_values
        self.alpha = self.sigma - self.beta
    
    
    @property
    def T(self):
        return self._T
    @T.setter
    def T(self, T):
        self._T = T
        
    @property
    def N(self):
        return self._N
    @N.setter
    def N(self, N):
        self._N = N
               
    @property
    def beta(self):
        return self._beta
    @beta.setter
    def beta(self, beta):
        self._beta = beta
        
    @property
    def sigma(self):
        return self._sigma
    @sigma.setter
    def sigma(self, sigma):
        self._sigma = sigma        
        
    @property
    def epsilon(self):
        return self._epsilon
    @epsilon.setter
    def epsilon(self, epsilon):
        self._epsilon = epsilon
          
    @property
    def t_values(self):
        return self._t_values
    @t_values.setter
    def t_values(self, t_values):
        self._t_values = t_values
        
    def laguerre(self, n, t):
        if (self.beta>=0) and (self.beta<=self.sigma):
            if n==0:
                return np.sqrt(self.sigma)*np.exp((-self.beta*t)/2)
            elif n==1:
                return np.sqrt(self.sigma)*(1-self.sigma*t)*np.exp((-self.beta*t)/2)
            else:
                l0=np.sqrt(self.sigma)*np.exp((-self.beta*t)/2)
                l1=np.sqrt(self.sigma)*(1-self.sigma*t)*np.exp((-self.beta*t)/2)
                for i in range(2, n+1):
                    li=((2*i-1-self.sigma*t)/i)*l1 - ((i-1)/i)*l0
                    l0, l1 = l1, li
                return li

        else:
            raise ValueError("Wrong data input! Beta should be in range [0;sigma]!")
        
    
    def tabulate_laguerre(self): 
        laguerre_val = {"t_val": self.t_values}  
        for n in range(self.N + 1):
            laguerre_val[f"L_{n}"] = [self.laguerre(n, t) for t in self.t_values]
        
        return pd.DataFrame(laguerre_val)
    
        
    def find_t_epsilon(self, n, epsilon=1e-3, max_T=100, steps=1000):
        t_values = np.linspace(0, max_T, steps)
        for t in t_values:
            if all(np.abs(self.laguerre(i, t)) < epsilon for i in range(n)):
                return t
        raise ValueError("No value t")

In [12]:
import numpy as np

# Функція Лагерра
def laguerre_function(t, n, alpha=2, beta=4):
    if n == 0:
        return np.sqrt(beta) * np.exp(-beta * t)
    elif n == 1:
        return np.sqrt(beta) * (1 - beta * t) * np.exp(-beta * t)
    else:
        Ln_2 = np.sqrt(beta) * np.exp(-beta * t)  # L_0
        Ln_1 = np.sqrt(beta) * (1 - beta * t) * np.exp(-beta * t)  # L_1
        for i in range(2, n + 1):
            Ln = ((2 * i - 1 - beta * t) * Ln_1 - (i - 1) * Ln_2) / i
            Ln_2, Ln_1 = Ln_1, Ln
        return Ln

# Наближене обчислення інтегралів
def laguerre_transform(f, n, alpha=2, beta=4, epsilon=1e-3, T=10):
    k_values = []
    for k in range(n + 1):
        # Метод прямокутників
        t_values = np.linspace(0, T, int(1 / epsilon))
        integrand = f(t_values) * laguerre_function(t_values, k, alpha, beta) * np.exp(-alpha * t_values)
        integral = np.sum(integrand) * (T / len(t_values))
        k_values.append(integral)
    return k_values

# Тестова функція f
def test_function(t):
    return np.sin(t)

# Обчислення для функції f при N=20
N = 20
coefficients = laguerre_transform(test_function, N)
print("Коефіцієнти Лагерра:", coefficients)


Коефіцієнти Лагерра: [np.float64(0.053983319630733266), np.float64(-0.01607073041241324), np.float64(-0.018595195481664262), np.float64(-0.01090237692197156), np.float64(-0.0051554645559359225), np.float64(-0.0021566591585600163), np.float64(-0.0008259950825643614), np.float64(-0.00029617203904814133), np.float64(-0.00010367707507298065), np.float64(-4.000112809196127e-05), np.float64(-2.1261202361348227e-05), np.float64(-1.6689390955699298e-05), np.float64(-1.6000543074154046e-05), np.float64(-1.6125081334668288e-05), np.float64(-1.6295891317719165e-05), np.float64(-1.6388728097758703e-05), np.float64(-1.6419949272879256e-05), np.float64(-1.6417839994875684e-05), np.float64(-1.640006663496434e-05), np.float64(-1.637522192315074e-05), np.float64(-1.6346957523715088e-05)]


In [ ]:
class IntegrateLaguerre(Laguerre):
    def __init__(self, laguerre_object, max_steps=10000):
        super().__init__(laguerre_object.T, laguerre_object.N, laguerre_object.beta, laguerre_object.sigma, laguerre_object.epsilon, laguerre_object.t_values)
        self.max_steps = max_steps

    def laguerre_transform(self, k, t_values):
        return np.array([self.laguerre(k, t) for t in t_values])

    def integrate(self, f, k):
        steps = 10
        prev_integral = 0
        delta = float("inf")
    
        while delta > self.epsilon:
            t_values = np.linspace(0, self.T, steps + 1)
            delta_t = self.T / steps
        
            lkt = self.laguerre_transform(k, t_values)
            ft = np.array([f.subs(sp.symbols('t'), t) for t in t_values])

            integrand = ft * lkt * np.exp(-1*self.alpha * t_values)
            integral = np.sum(integrand * delta_t) 
        
            delta = abs(integral - prev_integral)
            prev_integral = integral
        
            steps *= 2
            if steps > self.max_steps:
                raise RuntimeError("Максимальна кількість кроків перевищена. Інтеграл не збігається.")
    
        return prev_integral


In [4]:
class ReverseLaguerre(Laguerre):
    def __init__(self, laguerre_object, coefficients=None):
        super().__init__(laguerre_object.T, laguerre_object.N, laguerre_object.beta, laguerre_object.sigma, laguerre_object.epsilon, laguerre_object.t_values)
        self.coefficients = coefficients if coefficients is not None else []

    @property
    def coefficients(self):
        return self._coefficients

    @coefficients.setter
    def coefficients(self, coefficients):
        self._coefficients = coefficients

    def laguerre_coefficients(self, integr_obj, f):
        coeffs = []
        for k in range(self.N + 1): 
            coeff = integr_obj.integrate(f, k)
            coeffs.append(coeff)
        self.coefficients = coeffs 
        return coeffs 

    def reverse_laguerre(self, t):
        if len(self.coefficients) == 0:
            raise ValueError("Coefficients are not computed")
        
        result = 0
        for k in range(len(self.coefficients)):
            result += self.coefficients[k] * self.laguerre(k, t)
        return result

    def conditional_func(self, t_values=None):
        if t_values is None:
            t_values = np.linspace(0, self.T, 100)
        return np.array([self.reverse_laguerre(t) for t in t_values])


In [5]:
class TestLaguerre(unittest.TestCase):
    def setUp(self):
        self.t_values = np.linspace(0, 10, 100)
        self.laguerre_object = Laguerre(T=5, N=3, beta=1, sigma=2, epsilon=1e-3, t_values=self.t_values)
        self.integrate_object = IntegrateLaguerre(self.laguerre_object, max_steps=100000)
        self.reverse_object = ReverseLaguerre(self.laguerre_object)
        
    def test_laguerre_invalid_beta(self):
        """Test that laguerre raises ValueError with invalid beta"""
        self.laguerre_object.beta = -1
        with self.assertRaises(ValueError):
            self.laguerre_object.laguerre(1, 3)
            
    def test_laguerre_valid_params(self):
        """Test that laguerre calculation works with valid parameters"""
        self.laguerre_object.beta = 1
        self.laguerre_object.sigma = 2
        l0 = self.laguerre_object.laguerre(0, 1)
        self.assertIsNotNone(l0)
        l1 = self.laguerre_object.laguerre(1, 1)
        self.assertIsNotNone(l1)
        l2 = self.laguerre_object.laguerre(2, 1)
        self.assertIsNotNone(l2)
    
    def test_tabulate_laguerre(self):
        """Test tabulation of Laguerre polynomials"""
        self.laguerre_object.beta = 1
        self.laguerre_object.sigma = 2
        table = self.laguerre_object.tabulate_laguerre()
        self.assertIsInstance(table, pd.DataFrame)
        self.assertEqual(len(table), len(self.t_values))
        self.assertEqual(len(table.columns), self.laguerre_object.N + 2) 
    
    def test_find_t_epsilon(self):
        """Test finding epsilon value"""
        self.laguerre_object.beta = 0.5
        self.laguerre_object.sigma = 5
        try:
            t_eps = self.laguerre_object.find_t_epsilon(2, epsilon=0.5, max_T=50)
            self.assertGreater(t_eps, 0)
        except ValueError:
            t_eps = self.laguerre_object.find_t_epsilon(2, epsilon=10, max_T=50)
            self.assertGreater(t_eps, 0)
    
    def test_integrate_function(self):
        """Test integration of a symbolic function"""
        self.laguerre_object.beta = 1
        self.laguerre_object.sigma = 2
        t = sp.symbols('t')
        test_func = 1 + t
        result = self.integrate_object.integrate(test_func, 0)
        self.assertIsNotNone(result)
    
    def test_laguerre_coefficients(self):
        """Test calculation of Laguerre coefficients"""
        laguerre = Laguerre(T=5, N=2, beta=1, sigma=2, epsilon=1e-3, t_values=self.t_values)
        integrate = IntegrateLaguerre(laguerre, max_steps=100000)
        reverse = ReverseLaguerre(laguerre)
        
        t = sp.symbols('t')
        test_func = 1 + t
        coeffs = reverse.laguerre_coefficients(integrate, test_func)
        self.assertEqual(len(coeffs), laguerre.N + 1)
    
    def test_reverse_laguerre_no_coeffs(self):
        """Test that reverse_laguerre raises ValueError when coefficients are empty"""
        with self.assertRaises(ValueError):
            self.reverse_object.reverse_laguerre(1)
    
    def test_reverse_laguerre_with_coeffs(self):
        """Test reverse Laguerre transform with coefficients"""
        self.laguerre_object.N = 2
        self.laguerre_object.beta = 1
        self.laguerre_object.sigma = 2
        t = sp.symbols('t')
        test_func = 1 + t
        self.reverse_object.coefficients = [1.0, 0.5, 0.25] 
        result = self.reverse_object.reverse_laguerre(1)
        self.assertIsNotNone(result)
    
    def test_conditional_func(self):
        """Test conditional function calculation"""
        self.laguerre_object.N = 2
        self.laguerre_object.beta = 1
        self.laguerre_object.sigma = 2
        self.reverse_object.coefficients = [1.0, 0.5, 0.25]
        results = self.reverse_object.conditional_func(t_values=np.linspace(0, 2, 10))
        self.assertEqual(len(results), 10)


In [6]:
unittest.main(argv=[''], exit=False)

.........
----------------------------------------------------------------------
Ran 9 tests in 8.390s

OK


In [7]:
def parse_function(f_str):
    """Парсер стрінга у функу"""
    return sp.sympify(f_str)

In [8]:
class Visualization:
    @staticmethod
    def plot_laguerre(laguerre_obj, n, t):
        t_values = np.linspace(0, t, 100)
        y_values = [laguerre_obj.laguerre(n, t_i) for t_i in t_values]
        
        plt.figure(figsize=(8, 5))
        plt.plot(t_values, y_values, label=f'L_{n}(t)')
        plt.xlabel('t')
        plt.ylabel(f'L_{n}(t)')
        plt.title('Laguerre Polynomial')
        plt.legend()
        plt.grid()
        plt.show()
    
    @staticmethod
    def tabulate_laguere(laguerre_obj, n):
        df = laguerre_obj.tabulate_laguerre()
        display(df)
    
    @staticmethod
    def epsilon_vis(laguerre_obj, n):
        return laguerre_obj.find_t_epsilon(n)

    @staticmethod
    def integrate_vis(integrate_obj, n, f):
        return integrate_obj.integrate(f, n)

    @staticmethod
    def coef_vis(reverse_obj, integrate_obj, f):
        coefficients = reverse_obj.laguerre_coefficients(integrate_obj, f)
        print("Coefficients:", coefficients)
        return coefficients
    
    @staticmethod
    def reverse_plot(reverse_obj, t_values):
        if len(reverse_obj.coefficients) == 0:
            raise ValueError("Coefficients are not computed")
        
        y_values = [reverse_obj.reverse_laguerre(t) for t in t_values]
        
        plt.figure(figsize=(8, 5))
        plt.plot(t_values, y_values, label='Reverse Laguerre Transform')
        plt.xlabel('t')
        plt.ylabel('f(t)')
        plt.title('Reverse Laguerre Transform')
        plt.legend()
        plt.grid()
        plt.show()
    
    @staticmethod
    def output(laguerre_obj, integrate_obj, reverse_obj):
        dropdown = widgets.Dropdown(
            options=['plot_laguerre', 'tabulate_laguere', 'epsilon_vis', 'integrate_vis', 'coef_vis', 'reverse_plot'],
            description='Виберіть дію:',
        )

        output_area = widgets.Output()
        f_input = widgets.Text(value="sin(t)", description="Функція:")
        n_slider = widgets.IntSlider(value=3, min=0, max = 20, description="n:")
        sigma_slider = widgets.IntSlider(value=4, min = 0, max = 10, description="sigma:")
        T_slider = widgets.IntSlider(value=5,min = 1, max = 20, step = 1, description="T:")
        N_slider = widgets.IntSlider(value=5, min = 0, max = 20, step = 1, description="N:")
        beta_slider = widgets.IntSlider(value=2, min = 0, max = 10, description="beta:")
        t_slider = widgets.FloatSlider(value=5, min=0, max=20, description="t:")

        def update_plot(change=None):
            output_area.clear_output()
            f = parse_function(f_input.value)
            laguerre_obj.beta = beta_slider.value
            laguerre_obj.sigma = sigma_slider.value
            laguerre_obj.N = N_slider.value
            laguerre_obj.t_values = np.linspace(0, T_slider.value, 100)
            
            integrate_obj.beta = beta_slider.value
            integrate_obj.sigma = sigma_slider.value
            integrate_obj.N = N_slider.value
            integrate_obj.T = T_slider.value
            
            reverse_obj.beta = beta_slider.value
            reverse_obj.sigma = sigma_slider.value    
            reverse_obj.N = N_slider.value
            
            with output_area:
                if dropdown.value == "plot_laguerre":
                    Visualization.plot_laguerre(laguerre_obj, n_slider.value, t_slider.value)
                elif dropdown.value == "tabulate_laguere":
                    Visualization.tabulate_laguere(laguerre_obj, N_slider.value)
                elif dropdown.value == "epsilon_vis":
                    print(Visualization.epsilon_vis(laguerre_obj, n_slider.value))
                elif dropdown.value == "integrate_vis":
                    print(Visualization.integrate_vis(integrate_obj, n_slider.value, f))
                elif dropdown.value == "coef_vis":
                    Visualization.coef_vis(reverse_obj, integrate_obj, f)
                elif dropdown.value == "reverse_plot":
                    t_values = np.linspace(0, t_slider.value, 100)
                    Visualization.reverse_plot(reverse_obj, t_values)
                else:
                    print("Choose option!")

        dropdown.observe(update_plot, names='value')
        n_slider.observe(update_plot, names='value')
        T_slider.observe(update_plot, names='value')
        t_slider.observe(update_plot, names='value')
        beta_slider.observe(update_plot, names='value')
        sigma_slider.observe(update_plot, names='value')
        f_input.observe(update_plot, names='value')
        N_slider.observe(update_plot, names='value')

        display(f_input, N_slider, T_slider, n_slider, t_slider, sigma_slider, beta_slider, dropdown, output_area)

In [9]:
laguerre_object = Laguerre(0, 0, 0, 0, 10e3, np.linspace(0, 10, 100))
integral = IntegrateLaguerre(laguerre_object)
reversed_values = ReverseLaguerre(laguerre_object, [])

In [ ]:
Visualization.output(laguerre_object,integral,reversed_values)

Text(value='sin(t)', description='Функція:')

IntSlider(value=5, description='N:', max=20)

IntSlider(value=5, description='T:', max=20, min=1)

IntSlider(value=3, description='n:', max=20)

FloatSlider(value=5.0, description='t:', max=20.0)

IntSlider(value=4, description='sigma:', max=10)

IntSlider(value=2, description='beta:', max=10)

Dropdown(description='Виберіть дію:', options=('plot_laguerre', 'tabulate_laguere', 'epsilon_vis', 'integrate_…

Output()